In [1]:
!pip install --upgrade mlflow dagshub -q
!pip install pyngrok -q
!pip install --upgrade keras_tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## **Imports, mount drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction')

import os
import joblib
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tabulate import tabulate

from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column, row
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource, HoverTool, Legend

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Dropout,
    Input,
    BatchNormalization,
    Bidirectional
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau
)
from tensorflow.keras.regularizers import L1L2, l2
from tensorflow.keras.metrics import (
    MeanAbsolutePercentageError,
    RootMeanSquaredError,
    MeanAbsoluteError,
    MeanSquaredError
)

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    root_mean_squared_error
)

from config import *

from mlflow.models.signature import infer_signature
from bokeh.plotting import output_file, save

import dagshub
import mlflow

output_notebook()

Mounted at /content/drive


In [3]:
dagshub.init(repo_owner='bojte.csongor', repo_name='stock_market_prediction_thesis', mlflow=True)

mlflow.set_tracking_uri("https://dagshub.com/bojte.csongor/stock_market_prediction_thesis.mlflow")
mlflow.set_experiment(experiment_name="LSTM")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=7d9cd152-232c-4b4c-ad8a-ad51047494ff&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=938f6794c09600ba18ae1d404e4412635a8a3f73733977173ad4d0d5e07938dc




Output()

Accessing as bojte.csongor

Initialized MLflow to track repo "bojte.csongor/stock_market_prediction_thesis"

Repository bojte.csongor/stock_market_prediction_thesis initialized!

<Experiment: artifact_location='mlflow-artifacts:/63675e8552db4849830de73380a05984', creation_time=1743497830645, experiment_id='0', last_update_time=1743497830645, lifecycle_stage='active', name='LSTM', tags={}>

In [4]:
seed=0
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ["TF_DETERMINISTIC_OPS"] = str(seed)
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [5]:
def load_datasets(data_path, dataset_name):
    train_data = pd.read_csv(f"{data_path}/{dataset_name}_train.csv", index_col=0)
    val_data = pd.read_csv(f"{data_path}/{dataset_name}_val.csv", index_col=0)
    test_data = pd.read_csv(f"{data_path}/{dataset_name}_test.csv", index_col=0)

    try:
        train_data.index = pd.to_datetime(train_data.index, utc=True),
        val_data.index = pd.to_datetime(val_data.index, utc=True),
        test_data.index = pd.to_datetime(test_data.index, utc=True)
    except:
        pass

    scaler = joblib.load(f"{data_path}/{dataset_name}_scaler.joblib")
    return train_data, val_data, test_data, scaler

In [6]:
train_data, val_data, test_data, minmax_scaler = load_datasets(
    data_path=f"{PROCESSED_DATA_PATH}/minmax_split_first",
    dataset_name="normal_data"
)

train_data_last, val_data_last, test_data_last, minmax_last_scaler = load_datasets(
    data_path=f"{PROCESSED_DATA_PATH}/minmax_split_last",
    dataset_name="normal_data"
)

train_log_data, val_log_data, test_log_data, minmax_log_scaler = load_datasets(
    data_path=f"{PROCESSED_DATA_PATH}/minmax_split_first",
    dataset_name="log_data"
)

train_log_data_last, val_log_data_last, test_log_data_last, minmax_last_log_scaler = load_datasets(
    data_path=f"{PROCESSED_DATA_PATH}/minmax_split_last",
    dataset_name="log_data"
)

train_data_custom, val_data_custom, test_data_custom, custom_scaler = load_datasets(
    data_path=f"{PROCESSED_DATA_PATH}/custom_split_first",
    dataset_name="normal_data"
)

train_log_data_custom, val_log_data_custom, test_log_data_custom, custom_log_scaler = load_datasets(
    data_path=f"{PROCESSED_DATA_PATH}/custom_split_first",
    dataset_name="log_data"
)

In [7]:
def create_sequences(data, feature_cols, target_cols, sequence_length):
    features = data[feature_cols].values
    targets = data[target_cols].values

    n_samples = len(data) - sequence_length
    n_features = len(feature_cols)
    n_targets = len(target_cols)

    X = np.zeros((n_samples, sequence_length, n_features))
    y = np.zeros((n_samples, n_targets))

    for i in range(n_samples):
        X[i] = features[i:i+sequence_length]
        y[i] = targets[i+sequence_length-1]

    return X, y

In [8]:
def create_all_sequences(train,val,test, feature_cols, target_cols, sequence_length):
    X_train, y_train = create_sequences(
        data=train,
        feature_cols=feature_cols,
        target_cols=target_cols,
        sequence_length=sequence_length,
    )
    X_val, y_val = create_sequences(
        data=val,
        feature_cols=feature_cols,
        target_cols=target_cols,
        sequence_length=sequence_length,
    )
    X_test, y_test = create_sequences(
        data=test,
        feature_cols=feature_cols,
        target_cols=target_cols,
        sequence_length=sequence_length,
    )
    return X_train, y_train, X_val, y_val, X_test, y_test

In [9]:
feature_cols = ['Close']
target_cols = ['Target']
sequence_length = 10

X_train, y_train, X_val, y_val, X_test, y_test = create_all_sequences(
    train=train_data,
    val=val_data,
    test=test_data,
    feature_cols=feature_cols,
    target_cols=target_cols,
    sequence_length=sequence_length,
)

X_train_last, y_train_last, X_val_last, y_val_last, X_test_last, y_test_last = create_all_sequences(
    train=train_data_last,
    val=val_data_last,
    test=test_data_last,
    feature_cols=feature_cols,
    target_cols=target_cols,
    sequence_length=sequence_length,
)

X_train_log, y_train_log, X_val_log, y_val_log, X_test_log, y_test_log = create_all_sequences(
    train=train_log_data,
    val=val_log_data,
    test=test_log_data,
    feature_cols=feature_cols,
    target_cols=target_cols,
    sequence_length=sequence_length,
)

print("Normal Data, Split First:")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

print("\nNormal Data, Split Last:")
print(f"X_train_last shape: {X_train_last.shape}, y_train_last shape: {y_train_last.shape}")
print(f"X_val_last shape: {X_val_last.shape}, y_val_last shape: {y_val_last.shape}")
print(f"X_test_last shape: {X_test_last.shape}, y_test_last shape: {y_test_last.shape}")

print("\nLog Data, Split First:")
print(f"X_train_log shape: {X_train_log.shape}, y_train_log shape: {y_train_log.shape}")
print(f"X_val_log shape: {X_val_log.shape}, y_val_log shape: {y_val_log.shape}")
print(f"X_test_log shape: {X_test_log.shape}, y_test_log shape: {y_test_log.shape}")

Normal Data, Split First:
X_train shape: (1926, 10, 1), y_train shape: (1926, 1)
X_val shape: (405, 10, 1), y_val shape: (405, 1)
X_test shape: (405, 10, 1), y_test shape: (405, 1)

Normal Data, Split Last:
X_train_last shape: (1926, 10, 1), y_train_last shape: (1926, 1)
X_val_last shape: (405, 10, 1), y_val_last shape: (405, 1)
X_test_last shape: (405, 10, 1), y_test_last shape: (405, 1)

Log Data, Split First:
X_train_log shape: (1926, 10, 1), y_train_log shape: (1926, 1)
X_val_log shape: (405, 10, 1), y_val_log shape: (405, 1)
X_test_log shape: (405, 10, 1), y_test_log shape: (405, 1)


In [10]:
feature_cols_custom = ['Custom_Normalized']
target_cols_custom = ['Target']

X_train_custom, y_train_custom, X_val_custom, y_val_custom, X_test_custom, y_test_custom = create_all_sequences(
    train=train_data_custom,
    val=val_data_custom,
    test=test_data_custom,
    feature_cols=feature_cols_custom,
    target_cols=target_cols_custom,
    sequence_length=sequence_length,
)

X_train_log_custom, y_train_log_custom, X_val_log_custom, y_val_log_custom, X_test_log_custom, y_test_log_custom = create_all_sequences(
    train=train_log_data_custom,
    val=val_log_data_custom,
    test=test_log_data_custom,
    feature_cols=feature_cols_custom,
    target_cols=target_cols_custom,
    sequence_length=sequence_length,
)

In [11]:
def inverse_transform_simple(df, scaler, log_scaled=False):
    df = df.copy()
    inverse_scaled = scaler.inverse_transform(df)

    if log_scaled:
        inverse_scaled = np.exp(inverse_scaled)

    return inverse_scaled

In [12]:
def inverse_custom_normalize(normalized_value, last_value, index, n):
    if index == 0:
        return 0
    part1 = last_value * (index / n)
    sqrt_part = np.sqrt(index**2 + ((last_value * index) / n)**2)
    part2 = normalized_value * (sqrt_part / index)
    return part1 + part2

def add_first_value(data, first_value):
    return data + first_value

def inverse_transform_custom(arr, scaler, n, first_value, last_value, train_data,val_data,column_name='Custom_Normalized', log_scaled=False):
    start_index = len(train_data) + len(val_data)
    original_indices = np.arange(start_index, start_index + len(arr))

    inverse_minmax = scaler.inverse_transform(arr)[:,0]

    df_real = pd.DataFrame(inverse_minmax, columns=[column_name],
    index=original_indices)

    real_values = []

    for i, index in enumerate(df_real.index):
        real_value = inverse_custom_normalize(df_real.iloc[i, 0], last_value, index+sequence_length, n)
        real_values.append(real_value)

    real_values = add_first_value(np.array(real_values), first_value)

    if log_scaled:
        real_values = np.exp(real_values)

    return real_values

In [13]:
reference_values = pd.read_csv(f"{PROCESSED_DATA_PATH}/custom_split_first/normal_data_custom_scaler.csv")

first_value = reference_values['first_value'].iloc[0]
last_value = reference_values['last_value'].iloc[0]
last_index = reference_values['last_index'].iloc[0]


In [14]:
log_data_path = f"{PROCESSED_DATA_PATH}/custom_split_first/log_data_custom_scaler.csv"
raw_data_log = pd.read_csv(log_data_path)

first_value_log = raw_data_log['first_value'].iloc[0]
first_index_log = 0
last_value_log = raw_data_log['last_value'].iloc[0]
last_index_log = raw_data_log['last_index'].iloc[0]

In [15]:
def log_run_metadata(params: dict, tags: dict):
    for k, v in params.items():
        mlflow.log_param(k, v)
    for k, v in tags.items():
        mlflow.set_tag(k, v)

In [16]:
def evaluate_and_log_metrics(model, X_test, y_test, scaler, model_name, custom, log_scaled, train_data,val_data):
    if(NORM_TYPE == 'minmax_split_first_log'): log_scaled = True;
    y_pred = model.predict(X_test)

    if custom:
      y_pred_ext = np.hstack([y_pred, np.zeros_like(y_pred)])
      y_pred_real = inverse_transform_custom(y_pred_ext, scaler, last_index, first_value, last_value, train_data,val_data,log_scaled=log_scaled)

      y_test_ext = np.hstack([y_test, np.zeros_like(y_test)])
      y_test_real = inverse_transform_custom(y_test_ext, scaler, last_index, first_value, last_value, train_data,val_data,log_scaled=log_scaled)
    else:
      y_pred_ext = np.hstack([y_pred, np.zeros_like(y_pred)])
      y_pred_real = inverse_transform_simple(y_pred_ext, scaler,log_scaled=log_scaled)[:, 0].reshape(-1, 1)

      y_test_ext = np.hstack([y_test, np.zeros_like(y_test)])
      y_test_real = inverse_transform_simple(y_test_ext, scaler,log_scaled=log_scaled)[:, 0].reshape(-1, 1)

    metrics = evaluate_predictions(model_name, y_test_real, y_pred_real, should_print=True)

    mlflow.log_metric('mape', metrics['mape'])
    mlflow.log_metric('mse', metrics['mse'])
    mlflow.log_metric('mae', metrics['mae'])
    mlflow.log_metric('mpd', metrics['mpd'])
    mlflow.log_metric('rmse', metrics['rmse'])

    return y_test_real, y_pred_real

In [17]:
def get_callbacks(model_name, save_path):
    checkpoint_path = os.path.join(save_path)

    callbacks = [
        ModelCheckpoint(
            filepath=checkpoint_path,
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=0
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1,
            mode='min'
        ),
    ]

    return callbacks

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, LayerNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

def create_lstm_model(
    input_shape,
    layers,
    units,
    activation,
    loss,
    optimizer,
    use_layer_norm: bool = False,
    use_regularization: bool = False,
    dropout: float = 0.0
):
    assert layers == len(units), "Length of units array must match number of layers."

    model = Sequential()
    model.add(Input(shape=input_shape))

    for i in range(layers):
        return_seq = i < layers - 1

        lstm_kwargs = {
            "units": units[i],
            "activation": activation,
            "return_sequences": return_seq
        }

        if use_regularization:
            lstm_kwargs.update({
                "recurrent_regularizer": l2(),
                "activity_regularizer": l2(),
                "kernel_regularizer": l2(),
            })

        model.add(LSTM(**lstm_kwargs))

        if use_layer_norm:
            model.add(LayerNormalization())

        if dropout > 0.0:
            model.add(Dropout(dropout))

    model.add(Dense(1))

    model.compile(optimizer=optimizer, loss=loss)

    return model

In [50]:
from mlflow.models.signature import infer_signature
import time

def run_experiment_group(
    model_type: str,
    units: list[int],
    optimizer_config: dict,
    learning_rate: float,
    loss: str,
    activation: str,
    experiment_name,
    batch_size: int,
    epochs: int,
    sequence_length: int,
    X_train_data,
    y_train_data,
    X_val_data,
    y_val_data,
    X_test_data,
    y_test_data,
    scaler,
    feature_cols,
    norm_type: str,
    create_model_fn,
    use_layer_norm: bool = True,
    use_regularization: bool = True,
    dropout: float = 0.0,
    save_dir: str = CHECKPOINTS_PATH,
    custom=False,
    log_scaled=False,
    train_data=None,
    val_data=None,
    extra_name: str = "v1",
    num_runs: int = 1
):
    # tf.compat.v1.enable_eager_execution()
    # mlflow.keras.autolog(disable=True)

    # Generate the description from parameters
    description = generate_description(
        model_type=model_type,
        layers=len(units),
        units=units,
        activation=activation,
        norm=use_layer_norm,
        reg=use_regularization,
        dropout=dropout,
        seq_len=sequence_length,
        loss=loss,
        optimizer_name=optimizer_config["name"],
        extra=extra_name
    )

    for run_index in range(num_runs):
        with mlflow.start_run() as run:
            run_id = run.info.run_id
            model_name = f"{model_type}_{run_index}_{run_id}"
            checkpoint_path = os.path.join(save_dir, model_type, f"{model_name}.keras")

            model_dir = os.path.dirname(checkpoint_path)
            os.makedirs(model_dir, exist_ok=True)

            input_shape = X_train_data.shape[1:]

            # --- Logging setup ---
            run_group = f"{model_type}_{description}"
            mlflow.set_tag("mlflow.runName", model_name)
            mlflow.set_tag("architecture_type", description)
            mlflow.set_tag("run_group", run_group)
            mlflow.set_tag("run_index", run_index)
            mlflow.set_tag("experiment_name", experiment_name)

            log_run_metadata(
                params={
                    "normalization_method": norm_type,
                    "extra_name": extra_name,
                    "input_shape": input_shape,
                    "sequence_length": sequence_length,
                    "features": feature_cols,
                    "activation": activation,
                    "optimizer": optimizer_config['name'],
                    "learning_rate": learning_rate,
                    "lossfn": loss,
                    "layers": len(units),
                    "units": "_".join(map(str, units)),
                    "batch_size": batch_size,
                    "epochs": epochs,
                    "use_layer_norm": use_layer_norm,
                    "use_regularization": use_regularization,
                    "dropout": dropout
                },
                tags={
                    "model": model_type,
                    "architecture": "_".join(map(str, units)),
                    "description": description,
                    "sequence_length": sequence_length
                }
            )

            # --- Model creation & training ---
            model = create_model_fn(
                input_shape=input_shape,
                layers=len(units),
                units=units,
                activation=activation,
                loss=loss,
                optimizer=optimizer_config['create'](),
                use_layer_norm=use_layer_norm,
                use_regularization=use_regularization,
                dropout=dropout
            )

            callbacks = get_callbacks(model_name, checkpoint_path)

            history = model.fit(
                X_train_data, y_train_data,
                validation_data=(X_val_data, y_val_data),
                epochs=epochs,
                batch_size=batch_size,
                callbacks=callbacks,
                shuffle=False,
                verbose=0
            )

            # --- Load best model ---
            loaded_model = load_model(checkpoint_path)

            # --- Evaluation & logging ---
            y_test_real, y_pred_real = evaluate_and_log_metrics(
                loaded_model, X_test_data, y_test_data,
                scaler, model_name, custom=custom, log_scaled=log_scaled, train_data=train_data, val_data=val_data
            )

            print(y_test_real[-5:])

            # --- Artifact logging ---
            artifact_dir = os.path.join("artifacts", model_type, run_id)
            os.makedirs(artifact_dir, exist_ok=True)

            # Log model
            mlflow.keras.log_model(loaded_model, artifact_path="best_model")

            # Save model summary
            model_summary_path = os.path.join(artifact_dir, "model_summary.txt")
            with open(model_summary_path, "w") as f:
                loaded_model.summary(print_fn=lambda x: f.write(x + "\n"))

            # Save predictions plot
            fig = plot_predictions_bokeh(y_test_real, y_pred_real)
            pred_plot_path = os.path.join(artifact_dir, "predictions_plot.html")
            output_file(pred_plot_path)
            save(fig)

            # Save training plot
            training_plot = plot_training_history(history)
            history_plot_path = os.path.join(artifact_dir, "training_history_plot.html")
            output_file(history_plot_path)
            save(training_plot)

            # 🚀 Log all artifacts at once (fast!)
            mlflow.log_artifacts(artifact_dir)

            print(f"[✓] Run {run_index + 1}/{num_runs} complete — {model_name}")

relu - sigmoid - silu
adamax - lion
mse - huber


experiments:
- data_normalization: custom_log
- sequence length: 7,9,10,14,21,30 -> 10
- architecture: 80
- activation: relu
- loss: mse
- optimier: Lamb

In [ ]:
sequence_length

10

In [56]:
from keras import activations
from keras.optimizers import SGD, Adamax, Lamb, Lion, Adamax, Nadam

NORM_TYPE = "custom_log_split_first"

data_switch = {
    "minmax_split_first": {
        "data": (
        X_train, y_train,
        X_val, y_val,
        X_test, y_test
      ),
        "scaler": minmax_scaler,
        "log_scaled": False,
        "custom": False
    },
    "minmax_split_last": {
        "data": (
        X_train_last, y_train_last,
        X_val_last, y_val_last,
        X_test_last, y_test_last
      ),
        "scaler": minmax_last_scaler,
        "log_scaled": False,
        "custom": False
    },
    "minmax_split_first_log": {
        "data": (
        X_train_log, y_train_log,
        X_val_log, y_val_log,
        X_test_log, y_test_log
      ),
        "scaler": minmax_log_scaler,
        "log_scaled": True,
        "custom": False
    },
    "custom_split_first": {
        "data": (
            X_train_custom, y_train_custom,
            X_val_custom, y_val_custom,
            X_test_custom, y_test_custom
        ),
        "scaler": custom_scaler,
        "log_scaled": False,
        "custom": True,
        "first_value": reference_values['first_value'].iloc[0],
        "last_value" : reference_values['last_value'].iloc[0],
        "last_index" : reference_values['last_index'].iloc[0],
        "train_data": train_data_custom,
        "val_data": val_data_custom,
    },
    "custom_log_split_first": {
        "data": (
            X_train_log_custom, y_train_log_custom,
            X_val_log_custom, y_val_log_custom,
            X_test_log_custom, y_test_log_custom
        ),
        "scaler": custom_log_scaler,
        "log_scaled": True,
        "custom": True,
        "first_value": raw_data_log['first_value'].iloc[0],
        "last_value" : raw_data_log['last_value'].iloc[0],
        "last_index" : raw_data_log['last_index'].iloc[0],
        "train_data": train_log_data_custom,
        "val_data": val_log_data_custom,
    }
}

try:
    (
        X_train_data, y_train_data,
        X_val_data, y_val_data,
        X_test_data, y_test_data
    ) = data_switch[NORM_TYPE]['data']
    scaler = data_switch[NORM_TYPE]['scaler']
    log_scaled = data_switch[NORM_TYPE]['log_scaled']
    custom = data_switch[NORM_TYPE]['custom']

    if custom:
        first_value = data_switch[NORM_TYPE]['first_value']
        last_value = data_switch[NORM_TYPE]['last_value']
        last_index = data_switch[NORM_TYPE]['last_index']
        train_data = data_switch[NORM_TYPE]['train_data']
        val_data = data_switch[NORM_TYPE]['val_data']
except KeyError:
    raise ValueError(f"Unsupported normalization type: {NORM_TYPE}")

run_experiment_group(
    model_type="LSTM",
    num_runs=5,
    extra_name="loss_fn",
    experiment_name="loss_fn",
    units=[80],
    batch_size=32,
    epochs=150,
    optimizer_config={
        "name": "LosScaleOptimizer-Lamb",
        "create": lambda: tf.keras.optimizers.LossScaleOptimizer(Lamb(0.001))
    },
    learning_rate=0.001,
    loss="mse",
    activation="relu",
    use_layer_norm=True,
    use_regularization=True,
    dropout=0.0,
    sequence_length=sequence_length,
    feature_cols=feature_cols,
    norm_type=NORM_TYPE,
    custom=custom,
    log_scaled=log_scaled,
    scaler=scaler,
    create_model_fn=create_lstm_model,
    X_train_data=X_train_data,
    y_train_data=y_train_data,
    X_val_data=X_val_data,
    y_val_data=y_val_data,
    X_test_data=X_test_data,
    y_test_data=y_test_data,
    train_data=train_data,
    val_data=val_data,
)


Epoch 1: val_loss improved from inf to 1.16955, saving model to /content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/checkpoints/LSTM/LSTM_0_a711324f01ce45388e3183107374e782.keras

Epoch 2: val_loss did not improve from 1.16955

Epoch 3: val_loss improved from 1.16955 to 0.74470, saving model to /content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/checkpoints/LSTM/LSTM_0_a711324f01ce45388e3183107374e782.keras

Epoch 4: val_loss did not improve from 0.74470

Epoch 5: val_loss did not improve from 0.74470

Epoch 6: val_loss improved from 0.74470 to 0.67921, saving model to /content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/checkpoints/LSTM/LSTM_0_a711324f01ce45388e3183107374e782.keras

Epoch 7: val_loss did not improve from 0.67921

Epoch 8: val_loss did not improve from 0.67921

Epoch 9: val_loss improved from 0.67921 to 0.58014, saving model to /content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/checkpoints/LSTM/LSTM_0_a711324f01ce45388e31831

2025/04/09 20:11:27 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


[5867.08007812 5930.85009766 5974.06982422 6040.04003906 6037.58984375]


2025/04/09 20:11:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[✓] Run 1/5 complete — LSTM_0_a711324f01ce45388e3183107374e782
🏃 View run LSTM_0_a711324f01ce45388e3183107374e782 at: https://dagshub.com/bojte.csongor/stock_market_prediction_thesis.mlflow/#/experiments/0/runs/a711324f01ce45388e3183107374e782
🧪 View experiment at: https://dagshub.com/bojte.csongor/stock_market_prediction_thesis.mlflow/#/experiments/0

Epoch 1: val_loss improved from inf to 1.07751, saving model to /content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/checkpoints/LSTM/LSTM_1_c1dbd37862f74bcba51e4a51e4519263.keras

Epoch 2: val_loss improved from 1.07751 to 0.94627, saving model to /content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/checkpoints/LSTM/LSTM_1_c1dbd37862f74bcba51e4a51e4519263.keras

Epoch 3: val_loss improved from 0.94627 to 0.86592, saving model to /content/drive/MyDrive/Colab Notebooks/Stock_Market_Prediction/checkpoints/LSTM/LSTM_1_c1dbd37862f74bcba51e4a51e4519263.keras

Epoch 4: val_loss improved from 0.86592 to 0.78166, saving model

2025/04/09 20:13:45 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


[5867.08007812 5930.85009766 5974.06982422 6040.04003906 6037.58984375]


2025/04/09 20:13:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[✓] Run 2/5 complete — LSTM_1_c1dbd37862f74bcba51e4a51e4519263
🏃 View run LSTM_1_c1dbd37862f74bcba51e4a51e4519263 at: https://dagshub.com/bojte.csongor/stock_market_prediction_thesis.mlflow/#/experiments/0/runs/c1dbd37862f74bcba51e4a51e4519263
🧪 View experiment at: https://dagshub.com/bojte.csongor/stock_market_prediction_thesis.mlflow/#/experiments/0
🏃 View run LSTM_2_57c0df301cc64bd5a7bf93729c493eca at: https://dagshub.com/bojte.csongor/stock_market_prediction_thesis.mlflow/#/experiments/0/runs/57c0df301cc64bd5a7bf93729c493eca
🧪 View experiment at: https://dagshub.com/bojte.csongor/stock_market_prediction_thesis.mlflow/#/experiments/0


KeyboardInterrupt: 

In [20]:
def generate_description(
    model_type: str,
    layers: int,
    units: list[int],
    activation: str,
    norm: bool,
    reg: bool,
    dropout: float,
    seq_len: int,
    loss: str,
    optimizer_name: str,
    extra: str = ""
) -> str:
    desc = f"{model_type}_L{layers}_U{'-'.join(map(str, units))}_{activation}"
    if norm:
        desc += "_norm"
    if reg:
        desc += "_reg"
    if dropout > 0:
        desc += f"_drop{dropout}"
    desc += f"_seq{seq_len}_{loss}_{optimizer_name}"
    if extra:
        desc += f"_{extra}"
    return desc

In [ ]:
# run_id = "4f2aefb2c5614d30a6d715bcf152debe"
# loade_model = load_model(f"{CHECKPOINTS_PATH}/LSTM/{run_id}.keras")


# y_test_real, y_pred_real = evaluate_and_log_metrics(
#         loaded_model, X_test_data, y_test_data, scaler, model_name
# )

🏃 View run serious-duck-422 at: https://dagshub.com/bojte.csongor/stock_market_prediction_thesis.mlflow/#/experiments/0/runs/dbb6a10d78b4488b9f57596fd38fdfaa
🧪 View experiment at: https://dagshub.com/bojte.csongor/stock_market_prediction_thesis.mlflow/#/experiments/0


In [21]:
def calculate_mpd(y_true, y_pred):
    # Convert inputs to numpy arrays if they aren't already
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()

    # Calculate percentage deviations
    epsilon = 1e-7  # Avoid division by zero
    percentage_deviations = np.abs((y_true - y_pred) / (y_true + epsilon)) * 100

    # Find maximum deviation and its index
    max_deviation = np.max(percentage_deviations)
    max_deviation_idx = np.argmax(percentage_deviations)

    return {
        'mpd': max_deviation,
        'index': max_deviation_idx,
        'true_value': y_true[max_deviation_idx],
        'pred_value': y_pred[max_deviation_idx],
        'all_deviations': percentage_deviations
    }

In [22]:
def save_model_metrics(model_name, metrics, save_path):
    # Define column names
    columns = ['model_name', 'mape', 'mse', 'rmse', 'mae', 'mpd', 'mpd_index']

    # Convert metrics to DataFrame
    new_data = pd.DataFrame([{**{'model_name': model_name}, **metrics}])[columns]

    # Check if file exists
    if os.path.exists(save_path):
        df = pd.read_csv(save_path)

        # If model exists, update the row
        if model_name in df['model_name'].values:
            df.loc[df['model_name'] == model_name, columns[1:]] = new_data.iloc[:, 1:].values
        else:
            df = pd.concat([df, new_data], ignore_index=True)
    else:
        df = new_data

    # Save the updated DataFrame
    df.to_csv(save_path, index=False)
    print(f"Metrics saved to {save_path}")

In [23]:
def evaluate_predictions(model_name, y_true, y_pred, n_samples=None, should_print=False):
    # Flatten arrays if needed
    y_pred = y_pred.copy().flatten()
    y_true = y_true.copy().flatten()

    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    rmse = root_mean_squared_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    # Calculate MPD
    mpd_results = calculate_mpd(y_true, y_pred)
    mpd = mpd_results['mpd']
    mpd_index = mpd_results['index']
    true_value_at_mpd = mpd_results['true_value']
    pred_value_at_mpd = mpd_results['pred_value']
    percentage_deviations = mpd_results['all_deviations']

    if should_print:
        # Print metrics
        print("\nModel Performance Metrics:")
        print("-" * 50)
        print(f"MAPE: {mape:.4f}%")
        print(f"RMSE: {rmse:.8f}")
        print(f"MSE: {mse:.8f}")
        print(f"MAE: {mae:.8f}")
        print(f"MPD (Maximum Percentage Deviation): {mpd:.4f}%")

         # Print point of maximum deviation
        print(f"\nPoint of Maximum Deviation (Index {mpd_index}):")
        print("-" * 50)
        print(f"True Value: {y_true[mpd_index]:.6f}")
        print(f"Predicted Value: {y_pred[mpd_index]:.6f}")
        print(f"Absolute Difference: {abs(y_true[mpd_index] - y_pred[mpd_index]):.6f}")
        print(f"Percentage Deviation: {percentage_deviations[mpd_index]:.2f}%")

        # Print sample predictions
        if n_samples:
          print(f"\nFirst {n_samples} Predictions:")
          print("-" * 50)
          print("Index    True Value    Predicted    Difference    % Deviation")
          print("-" * 65)
          for i in range(min(n_samples, len(y_true))):
              diff = y_true[i] - y_pred[i]
              dev = percentage_deviations[i]
              print(f"{i:<8d} {y_true[i]:11.6f}  {y_pred[i]:11.6f}  {diff:11.6f}  {dev:11.2f}%")

    # save_model_metrics(model_name, {
    #     'mape': mape,
    #     'mse': mse,
    #     'rmse': rmse,
    #     'mae': mae,
    #     'mpd': mpd,
    #     'mpd_index': mpd_index,
    # }, f"{METRICS_PATH}/model_metrics.csv")

    return {
        'mape': mape,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'mpd': mpd,
        'mpd_index': mpd_index,
        'percentage_deviations': percentage_deviations}

In [24]:
def plot_training_history(history):
    output_notebook()

    # Create data sources
    epochs = list(range(1, len(history.history['loss']) + 1))

    # Ensure values are positive for log scale (add small epsilon if needed)
    epsilon = 1e-10
    train_loss = [max(val, epsilon) for val in history.history['loss']]
    val_loss = [max(val, epsilon) for val in history.history['val_loss']]

    source_loss = ColumnDataSource(data={
        'epoch': epochs,
        'train_loss': train_loss,
        'val_loss': val_loss
    })

    p1 = figure(title='Model Loss Over Time (Log Scale)',
               x_axis_label='Epoch',
               y_axis_label='Loss (log)',
               width=600, height=400,
               y_axis_type="log")

    # Add hover tool
    hover_loss = HoverTool(tooltips=[
        ('Epoch', '@epoch'),
        ('Training Loss', '@train_loss{0.000}'),
        ('Validation Loss', '@val_loss{0.000}')
    ])
    p1.add_tools(hover_loss)

    # Plot loss lines
    l1 = p1.line('epoch', 'train_loss', line_color=Category10[3][0],
                 line_width=2, source=source_loss, legend_label='Training Loss')
    l2 = p1.line('epoch', 'val_loss', line_color=Category10[3][1],
                 line_width=2, source=source_loss, legend_label='Validation Loss')

    # Configure legends
    for p in [p1]:
        p.legend.click_policy = "hide"
        p.legend.location = "top_right"
        p.grid.grid_line_alpha = 0.3

    # Show plots
    show(p1)
    return p1

In [25]:
def plot_predictions_bokeh(y_test, y_pred, n_samples=None):
    output_notebook()

    if n_samples is None:
        n_samples = len(y_test)
    else:
        n_samples = min(n_samples, len(y_test))

    # Prepare data
    x_range = list(range(n_samples))
    source = ColumnDataSource(data={
        'index': x_range,
        'actual': y_test[:n_samples],
        'predicted': y_pred[:n_samples],
        'error': y_test[:n_samples] - y_pred[:n_samples]
    })

    # Create time series plot
    p1 = figure(title='Actual vs Predicted Values',
                x_axis_label='Sample Index',
                y_axis_label='Value',
                width=800, height=400)

    # Add hover tool
    hover = HoverTool(tooltips=[
        ('Index', '@index'),
        ('Actual', '@actual{0.000}'),
        ('Predicted', '@predicted{0.000}'),
        ('Error', '@error{0.000}')
    ])
    p1.add_tools(hover)

    # Plot lines
    l1 = p1.line('index', 'actual', line_color=Category10[3][0],
                 line_width=2, source=source, legend_label='Actual')
    l2 = p1.line('index', 'predicted', line_color=Category10[3][1],
                 line_width=2, source=source, legend_label='Predicted')

    # Show plots
    show(row(p1))

    return p1;